In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun Aug 18 11:22:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   42C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
path = '/content/drive/MyDrive/project/data/'
base_directory = path # Your Base Directory

# 설명

## Question - Answering with Retrieval

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

## Mount/Login

구글 드라이브를 마운트하고 허깅페이스에 로그인
- 이때 허깅페이스 토큰은 kdt3 그룹에 대해 읽기/쓰기 권한이 있는 토큰이어야 함

## Download Library
필요/사용 라이브러리 다운로드
이때 버전 문제로 설치를 한 뒤 세션을 한번 재시작해줘야 합니다
<br>(그리고 세션 완전히 끊기면 다운로드 후 재시작을 다시 해줘야...)

## Import Library
한번 재시작했으면 위 과정 없이 Import만 실행해주면 됩니다

## Vector DB
문서를 여러 조각(chunk)로 나누고, 임베딩 유사도를 통해 관련 조각을 찾을 수 있게 DB화하는 함수들이 정의되어 있습니다.

## DB 생성
Vector DB에서 정의된 함수들로 문서 DB를 만들어줍니다.<br><br>
이때 Train과 Test를 한번에 하려고 하면 코랩이 터질 확률이 높으므로 Train하고 Create Dataset까지 실행해 업로드 한 뒤 재시작해서 램을 비우고 Test를 하는 것이 좋습니다.<br> 또한 문서 임베딩을 어떤 모델로 할지 인자로 넘겨줄 수 있습니다

## Create Dataset
DB 생성에서 만든 db와 데이터 dataframe을 사용해 HuggingFace 데이터셋 생성 후 업로드

## Fine-Tuning
학습 데이터셋으로 모델에 대한 파인튜닝 진행 후 Huggingface에 업로드<br>
4비트 양자화 LoRA로 파인튜닝<br>
기반 모델 또는 넣어줄때 사용할 프롬프트, 학습 관련 하이퍼파라미터 수정 가능

## Langchain 을 이용한 추론
모델을 사용한 추론


## 실행
### 기본
Mount/Login -> Download Library -> 재시작 (처음 1번)
Mount/Login -> Import Library (이후)

### 데이터셋 만들기
기본 -> Vector DB -> DB 생성 -> Create Dataset에서 첫 셀 + Train/Valid/Test 중 해당하는 셀

### 모델 학습하기
기본 -> Fine-Tuning(업로드할 위치, 데이터셋 위치, 모델 링크 확인 필수)

### 학습된 모델로 추론하기
기본 -> Langchain을 이용한 추론(모델 링크, 데이터셋 위치 확인) -> Submission(저장할 파일명 확인)

# Mount/Login

In [ ]:
from huggingface_hub import login

login()

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Download Library

In [ ]:
!apt-get install tesseract-ocr
!apt-get install poppler-utils

!pip install orjson==3.10.6

!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install transformers[torch] -U

!pip install datasets
!pip install langchain
!pip install langchain_community
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-gpu
!pip install peft
!pip install trl
!pip install unstructured pdfminer.six
!pip install pillow-heif
!pip install unstructured_inference
!pip install unstructured_pytesseract
!pip install pikepdf pypdf


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Looking in indexes: https://pypi.org/simple/
  Using cached unstructured_inference-0.7.36-py3-none-any.whl.metadata (5.9 kB)
  Using cached layoutparser-0.3.4-py3-none-any.whl.metadata (7.7 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached onnx-1.16.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached onnxruntime-1.19.0-cp310-cp310-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.3 kB)
  Using cached timm-1.0.8-py3-none-any.whl.metadata (53 kB)
  Using

# Import Library

In [ ]:
import os
import unicodedata

import torch
import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF

from langchain.document_loaders.parsers.pdf import PDFPlumberParser


from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig
)
from accelerate import Accelerator

# peft
from peft import prepare_model_for_kbit_training
from peft import PeftModel
from peft import LoraConfig, get_peft_model


# Langchain 관련
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel
from langchain.schema.output_parser import StrOutputParser

# PDF 로딩/청크화 관련
from langchain.document_loaders.parsers.pdf import PDFPlumberParser
from langchain.document_loaders.pdf import PDFPlumberLoader
from langchain.document_loaders import UnstructuredPDFLoader

from unstructured.cleaners.core import clean_extra_whitespace, clean, clean_non_ascii_chars

import pdfplumber


# Vector DB

In [ ]:
from operator import itemgetter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from unstructured.cleaners.core import clean_extra_whitespace, clean, clean_non_ascii_chars


# 단어가 표안에 들어있는지 확인하는 함수
def check_bboxes(word, table_bbox):
    """
    Check whether word is inside a table bbox.
    """
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

# 불릿포인트 제거용 함수
def remove_bulletpoints(text):
    cleaned_text = text
    for symbol in ['ㅇ','-','□', '※', '▸','∙','●','☞','■','','']:
        cleaned_text = cleaned_text.replace(symbol, f" ")
    for symbol in ['·']:
        cleaned_text = cleaned_text.replace(symbol, "/")
    return cleaned_text

# 숫자 심볼 숫자로 변환
def replace_num_symbols_with_number(text):
    cleaned_text = text
    for idx, symbol in enumerate(['①', '②', '③', '④', '⑤', '⑥', '⑦', '⑧', '⑨', '⑩', '⑪', '⑫', '⑬', '⑭', '⑮']):
        cleaned_text = cleaned_text.replace(symbol, f"{idx+1})")
    return cleaned_text

In [ ]:
import pdfplumber
from operator import itemgetter

def clean_string(text):
    text_string = clean(text, dashes=True,trailing_punctuation=True, bullets=True)
    text_string = replace_num_symbols_with_number(text_string)
    text_string = remove_bulletpoints(text_string)
    return text_string

def clean_table(text):
    text_string = replace_num_symbols_with_number(text)
    text_string = remove_bulletpoints(text_string)
    return text_string

# 원함수
def process_pdf(file_path, chunk_size=512, chunk_overlap=32): #chunk_size256 -> 512로 변경
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    # doc = parser.load(file_path)
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += clean_table(page.get_text())
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path="intfloat/multilingual-e5-small"):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory, model_path = "intfloat/multilingual-e5-small"):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")

        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks, model_path = model_path)

        # Retriever 생성
        retriever = db.as_retriever(search_type="mmr",
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        # retriever = db.as_retriever(search_type="similarity",
        #                             search_kwargs={'k': 3, 'fetch_k': 8})


        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases



# 증강데이터 생성

In [ ]:
# 무작위 삭제 random_deletion
aug_rd = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/random_deletion.csv')

# 무작위 스왑 random_swap
aug_rs = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/random_swap.csv')

# 동의어 대체 synonym_replacement
aug_sr = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/synonym_replacement.csv')

# 무작위 삽입 random_insert
aug_ri = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/random_insertion.csv')

train_df = pd.concat([train_df, aug_rd, aug_rs, aug_sr, aug_ri], ignore_index=True)

In [ ]:
aug_bi = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/bert_insert.csv', index_col=0)
aug_rs = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/random_swap.csv', index_col=0)
aug_ri = pd.read_csv('/content/drive/MyDrive/project/data/augment_data/random_insertion.csv', index_col=0)

# DB 생성

In [ ]:
# 증강 데이터 훈련시킬시 아래코드만 실행
train_df = pd.concat([train_df, aug_bi, aug_rs, aug_ri], ignore_index=True)
# 증강하지 않은 원래 데이터 훈련시킬시 아래코드만 실행
train_df = pd.read_csv(f'{path}train.csv')

In [ ]:
test_df = pd.read_csv(f'{path}test.csv')

In [ ]:
model_path="intfloat/multilingual-e5-base"

train_db = process_pdfs_from_dataframe(train_df, base_directory, model_path=model_path)
test_db = process_pdfs_from_dataframe(test_df, base_directory, model_path=model_path)

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

Processing 중소벤처기업부_혁신창업사업화자금(융자)...


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/179k [00:00<?, ?B/s]

...

Processing PDFs:  78%|███████▊  | 7/9 [01:02<00:17,  8.96s/it]

Processing 「FIS 이슈&포커스」 22-2호 《재정성과관리제도》...


Processing PDFs:  89%|████████▉ | 8/9 [01:11<00:08,  8.81s/it]

Processing 「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...


Processing PDFs: 100%|██████████| 9/9 [01:21<00:00,  9.01s/it]


# Create Dataset

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

def make_dataset(df, pdf_databases):
    dataset = dict()
    dataset['context'] = list()
    dataset['question'] = list()
    dataset['answer'] = list()
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Making"):
        # 소스 문자열 정규화
        source = normalize_string(row['Source'])
        question = row['Question']
        if 'Answer' in df.columns:
            answer = row['Answer']

        # 정규화된 키로 데이터베이스 검색
        normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
        retriever = normalized_keys[source]['retriever']

        context = format_docs(retriever.invoke(question))

        dataset['context'].append(context)
        dataset['question'].append(question)
        if 'Answer' in df.columns:
            dataset['answer'].append(answer)
        else:
            dataset['answer'].append('')
    return dataset

In [ ]:
push_url = "kdt3/DACON-QA-augment_ver2"

## Train 데이터 생성 & 업로드

In [ ]:
from datasets import Dataset
train_dataset = make_dataset(train_df, train_db)
train_dataset = Dataset.from_dict(train_dataset)
train_dataset.push_to_hub(push_url, private=True, split='train')


Making: 100%|██████████| 1596/1596 [00:26<00:00, 60.04it/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-augment_ver2/commit/147a2e9ed5adf60b5366b808edeead5cd5613b92', commit_message='Upload dataset', commit_description='', oid='147a2e9ed5adf60b5366b808edeead5cd5613b92', pr_url=None, pr_revision=None, pr_num=None)

## Valid 데이터 생성 & 업로드

In [ ]:
from datasets import Dataset
valid_dataset = make_dataset(train_df.iloc[:100], train_db)
valid_dataset = Dataset.from_dict(valid_dataset)
valid_dataset.push_to_hub(push_url, private=True, split='valid')

Making: 100%|██████████| 100/100 [00:01<00:00, 59.37it/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/349 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-augment_ver2/commit/28c6ed42d6d0fb9c25d8e9a2366195371e6c3380', commit_message='Upload dataset', commit_description='', oid='28c6ed42d6d0fb9c25d8e9a2366195371e6c3380', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
for i in range(50,60):
    print(f"question: {valid_dataset['question'][i]}")
    print(f"context: {valid_dataset['context'][i]}")

question: 관리재정수지와 통합재정수지의 차이점은 무엇인가?
context: 55
2024년도 나라살림
제2부
2024년도 예산안 시정연설 및 제안설명
제1부
2024년도 예산 및 기금운용계획
제3부
참고 | 관리재정수지란 ?
관리재정수지는 통합재정수지 중 사회보장성기금수지를 제외한 재정수지를 말한다. 
사회보장성기금은 국민연금, 사학연금, 산업재해보상보험기금과 고용보험기금을 포함한다.
지속적으로 증가하고 있는 사회보장성기금의 흑자는 연금 등 장래 지급에 대비하여 적립되고
등을 중복 계상하여 실제의 재정수입 전체보다 커지는 문제가 발생한다. 총수입은 중앙 
재정의 실제 수입규모를 파악하기 위해 회계/기금간 내부거래 등을 제외하고 산출하는 
것으로 사실상 IMF가 국제적 비교를 위한 공통 기준으로 제시하는 통합재정 작성방식과 
거의 같은 개념이다. 
  총수입 = 일반회계수입 + 특별회계수입 + 기금수입   내부거래   보전거래
여러 기업으로 구성된 그룹이 그룹전체의 규모를 파악하기 위해 기업간 내부거래를
통합재정수지 △0.6%(△13.1조원) 및 관리재정수지 △2.6%(△58.2조원)에 비해 일부 
악화되었다. 어려운 세입여건에도 불구하고, 재정수지 악화를 최소화하였다.
정부는 약자복지, 역동경제 구현, 국민안전 등 재정의 필수적 역할은 지속하면서도, 
재정의 지속가능성을 유지하기 위한 재정건전성 관리노력을 병행해 나감으로써 
2025년부터 관리재정수지 적자폭을 GDP 대비 △3% 이내로 관리할 계획이다.
[표 4 1] 재정수지
(단위: 조원)
구 분

question: 청년층을 위한 주거비용 지원 정책과 산업단지를 청년이 선호하는 환경으로 전환하기 위한 프로젝트에 대해 설명해주세요.
context: 제1부
2024년도 예산 및 기금운용계획
제3부
먼저 주거비용 지원을 위해 당초 ’23년 종료 예정이던 월 20만원의 청년 월세 한시 
특별지원을 1년 연장 지원한다. 또한 청년 대상 공공분양(5.3 
→ 
6.7만호) 및 공공 
임대(5.2 
→ 
5.7만

## Test 데이터 생성 & 업로드

In [ ]:
from datasets import Dataset
test_dataset = make_dataset(test_df, test_db)
test_dataset = Dataset.from_dict(test_dataset)
test_dataset.push_to_hub(push_url, private=True, split='test')

Making: 100%|██████████| 98/98 [00:01<00:00, 56.15it/s]


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/449 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/kdt3/DACON-QA-augment_ver2/commit/db10eae79c532276519d7132ba661db07295ea37', commit_message='Upload dataset', commit_description='', oid='db10eae79c532276519d7132ba661db07295ea37', pr_url=None, pr_revision=None, pr_num=None)

# Fine-Tuning

In [ ]:
# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


model_id = "rtzr/ko-gemma-2-9b-it"

# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False
# tokenizer.padding_side="right"

# 모델 로드 및 양자화 설정 적용
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.5k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
#데이터셋 로드
from datasets import load_dataset

# dataset_url = "kdt3/DACON-QA-base-table-preprocessed-v2"
dataset_url = "kdt3/DACON-QA-augment_ver2"

train_dataset = load_dataset(dataset_url)

Generating train split:   0%|          | 0/1596 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/98 [00:00<?, ? examples/s]

In [ ]:
# 문맥 잘못됐나 확인 - 문맥 길이 체크
import numpy as np
np.argmax([len(x) for x in train_dataset['train']['context']])

5

In [ ]:
#예시
i= 0#367

print(train_dataset['train']['question'][i])
print('--------')
print(train_dataset['train']['context'][i])
print('--------')
print(train_dataset['train']['answer'][i])

2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?
--------
2. 재정수입
3. 재정지출
4. 지방재정 조정
5. 총사업비 관리대상사업
6. 계속비 대상사업
주요 재정통계
 
Ⅰ.
2
01 재정체계
 중앙정부 재정체계는 예산(일반･특별회계)과 기금으로 구분되며, 2024년 기준으로 일반회계 1개, 특별회계 
21개, 기금 68개로 구성
 2024년 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7조원으로 구성
 2024년 기금 지출은 49개 사업성기금 81.2조원, 6개 사회보험성기금 107.1조원, 5개 계정성기금 
30.1조원으로 구성
[그림 1 1] 재정지출 구조(2024년 예산 총지출 기준)
주: 괄호 안은 총계 기준 예산액을 의미
자료: 디지털예산회계시스템
2024 주요 재정통계 | 2024 Fiscal Statistics
Ⅰ. 주요재정통계
Ⅱ. 국제통계
부록
Ⅲ. 분야별 재정지출
Ⅰ. 주요재정통계
3
 예산은 ｢국가재정법｣에 근거해 정부가 편성하고 국회가 심의･의결로 확정한 재정지출계획을 의미하며, 
일반회계와 특별회계로 구분
4.0
6.6
11.2
0.4
2.6
11.6
 
 
(1.0)
(1.5)
(2.3)
(0.1)
(0.4)
(1.7)
( )
( )
합계
406.6
434.1
485.1
549.9
601.0
682.4
638.7
656.6
(100.0)
(100.0)
(100.0)
(100.0)
(100.0)
(100.0)
(100.0)
(100.0)
주1) 자치단체교부금 정산 등
자료: 디지털예산회계시스템
2024 주요 재정통계 | 2024 Fiscal Statistics
Ⅰ. 주요재정통계
Ⅱ. 국제통계
부록
Ⅲ. 분야별 재정지출
Ⅰ. 주요재정통계
31
 2024년 기준, 62개 중앙관서의 소관별 지출은 보건복지부 122.4조원(18.6%), 교육부 
95.8조원(14.6%), 행정안전부 72.4조원(11.0%), 국토교통부 60.9조원(9.3%) 등의 순임
[표 3 13] 중앙관서별 최근 5년 총지출 추이
(단위: 

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [ ]:
# # 학습 프롬프트 - 추론 프롬프트와 통일하는 것이 좋다고 함

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['question'])):

#         # prompt = f"###문서: {example['context'][i]} \n ###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         # prompt = f"당신은 재정관련 문서를 읽고 질문에 대해 답변해주는 AI도우미입니다. 주어진 문서에서 질문에 대한 답변을 찾아 주세요. 한 문장으로 답해주세요. 부연 설명을 하지 마세요. 질문을 반복하지 마세요. ###문서: {example['context'][i]} \n ###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         context = example['context'][i]
#         question = example['question'][i]
#         answer = example['answer'][i]
#         prompt = f"""
#     다음 정보를 바탕으로 질문에 답하세요:
#     {context}

#     질문: {question}

#     주어진 질문에만 답변하세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
#     답변: {answer}
#     """
#         # prompt = f"###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         output_texts.append(prompt)
#     return output_texts

# response_template = '답변:'

In [ ]:
# # 학습 프롬프트 - 추가본2, 영어버전이 추가본1

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['question'])):

#         # prompt = f"###문서: {example['context'][i]} \n ###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         # prompt = f"당신은 재정관련 문서를 읽고 질문에 대해 답변해주는 AI도우미입니다. 주어진 문서에서 질문에 대한 답변을 찾아 주세요. 한 문장으로 답해주세요. 부연 설명을 하지 마세요. 질문을 반복하지 마세요. ###문서: {example['context'][i]} \n ###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         context = example['context'][i]
#         question = example['question'][i]
#         answer = example['answer'][i]
#         prompt = f"""
#         ###지시###
#         지시:
#         1. 반드시 다음의 정보를 바탕으로 질문에 답변하라: {context}
#         2. 반드시 한국말로 대답하시오.

#         ###예시###
#         예시1:
#         '장애인 고용장려금은 2023년 대비 2024년에 어떻게 변화했는가?'
#         (답변:'장애인 고용장려금은 2023년에 2,977억원이었고, 2024년에 3,234억원으로 증가했다.')

#         예시2:
#         '어떤 국가들을 대상으로 인도적 지원을 확대할 것이며, 어떤 위험에 노출된 국가들을 주로 지원할 계획인가요?'
#         (답변:'우크라이나 등 분쟁지역과, 기후변화·식량위기 등 위험에 노출된 취약국을 대상으로 인도적 지원을 확대할 것이다.')


#         ###질문###
#         {question}

#         ###대답###
#         답변: {answer}
#         """
#         # prompt = f"###질문: {example['question'][i]} \n ###답변: {example['answer'][i]}"
#         output_texts.append(prompt)
#     return output_texts

# response_template = '답변:'

In [ ]:
# # # 학습 프롬프트 - 추가본3

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['question'])):

#         context = example['context'][i]
#         question = example['question'][i]
#         answer = example['answer'][i]

#         prompt = f"""
#         ### Role ###
#         You are an AI assistant who provides accurate and concise 'answers' to 'questions' in financial documents.
#         Generate 'answers' to 'questions' based on the content of the given 'Document'.

#         ### Document ###
#         {context}

#         ### Question ###
#         {question}

#         ### Answer ###
#         response: {answer}
#         """

#         output_texts.append(prompt)

#     return output_texts

# response_template = 'response:'

In [ ]:
# # 학습 프롬프트 - 추가본4

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):

        context = example['context'][i]
        question = example['question'][i]
        answer = example['answer'][i]

        prompt = f"""
        Read the following document carefully and answer the question.
        nDocument: {context}
        nQuestion: {question}
        Answer: {answer}
        """
        output_texts.append(prompt)

    return output_texts

response_template = 'Answer:'

In [ ]:
# 메트릭 계산 함수인데 왠지 안쓰인다...
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [ ]:
import transformers
from datetime import datetime
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM

# 로컬에 모델 저장하고 싶은 경우 이름 지정
project = "financeQA-prompt2_ver4"
base_model_name = "gemma3"
run_name = base_model_name + "_" + project
output_dir = path + run_name

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset['train'],
    eval_dataset = train_dataset['valid'],
    compute_metrics=compute_metrics,
    args=transformers.TrainingArguments(
        do_eval=True,
        output_dir=output_dir,
        warmup_ratio=0.05,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        gradient_checkpointing=True,
        num_train_epochs = 1,
        learning_rate=2e-4,
        fp16=True,
        optim="paged_adamw_8bit",
        logging_strategy='epoch',
        logging_dir=output_dir+"/logs",

        # save_strategy="steps",
        # save_steps=25,

    ),
    formatting_func=formatting_prompts_func, # 프롬프트 처리하기 위해 필요
    data_collator=DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer, mlm=False), #모델에게 답변생성에만 집중하도록 함
)

model.config.use_cache = False
trainer.train()

trained_model = (trainer.model.module if hasattr(trainer.model, "module") else trainer.model)

#로컬에 저장할 경우
# trained_model.save_pretrained(f"{output_dir}/saved_model")

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:289: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1596 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:408: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
It is strongly recommended to train Gemma2 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differ

Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.19 GiB. GPU 

In [ ]:
# # 모델 업로드
trained_model.push_to_hub("financeQA-prompt2_ver4", private=True)

NameError: name 'trained_model' is not defined

In [ ]:
# ## 로컬에 저장된 것 허깅페이스에 업로드

# # 모델 디렉토리 경로
# output_dir = "/content/drive/MyDrive/project/data/gemma3_financeQA-prompt2/saved_model"

# # 업로드할 저장소 이름
# model_name = "dacon-gemma2-context-alltext-prompt2"
# repo_id = f"harry2341/{model_name}"

# # Hugging Face API 객체 생성
# api = HfApi()

# # 디렉토리 업로드
# upload_folder(
#     folder_path=output_dir,
#     repo_id=repo_id,
#     repo_type="model",
#     token=HfFolder.get_token()
# )

adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/harry2341/dacon-gemma2-context-alltext-aug5to8/commit/1762c4410ed32a47b4a772461a4c624a677b2ca6', commit_message='Upload folder using huggingface_hub', commit_description='', oid='1762c4410ed32a47b4a772461a4c624a677b2ca6', pr_url=None, pr_revision=None, pr_num=None)

# Langchain 을 이용한 추론

In [ ]:
# HuggingFacePipeline 객체 생성


# 4비트 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 모델 ID

model_id = "rtzr/ko-gemma-2-9b-it"
peft_model_id = "harry2341/financeQA-prompt2_ver4"


# 토크나이저 로드 및 설정
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.use_default_system_prompt = False
# tokenizer.padding_side="right"

# 모델 로드 및 양자화 설정 적용
trained_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True)

#Fine-Tune 한 LoRA 어댑터 불러오기
trained_model.load_adapter(peft_model_id)


text_generation_pipeline = pipeline(
    model=trained_model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    max_new_tokens=200,
    # repetition_penalty=1.5,
    eos_token_id = tokenizer.eos_token_id,
    pad_token_id = tokenizer.pad_token_id
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/725 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.05G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(


In [ ]:
#데이터셋 로드
from datasets import load_dataset

# dataset_url = "kdt3/DACON-QA-base-table-preprocessed-v2"
dataset_url = "kdt3/DACON-QA-augment_ver2"

dataset = load_dataset(dataset_url)

In [ ]:
# 검증 데이터 쓸지, 테스트 데이터 쓸지
valid_dataset = dataset['valid']
test_dataset = dataset['test']

eval_dataset = test_dataset
# eval_dataset = valid_dataset

In [ ]:
# 그냥 GPU 메모리 확인용
!nvidia-smi

Fri Aug 16 07:55:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              51W / 400W |   8743MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [ ]:
from tqdm import trange

def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context


# 결과를 저장할 리스트 초기화
results = []

# Dataset 각 행에 대해 처리
for idx in trange(len(eval_dataset['question']), desc="Answering Questions"):
    # if idx > 176:
    #     break

    #질문
    question = eval_dataset['question'][idx]

    #컨텍스트(문서)
    context = eval_dataset['context'][idx]
    # context = ""


    # RAG 체인 구성
    # template = "당신은 재정관련 문서를 읽고 질문에 대해 답변해주는 AI도우미입니다. 주어진 문서에서 질문에 대한 답변을 찾아 주세요. 한 문장으로 답해주세요. 부연 설명을 하지 마세요. 질문을 반복하지 마세요. ###문서: {context} \n ###질문: {question} \n ###답변: "

    # template = "당신은 재정전문 AI입니다. 질문에 한국어로 간결하고 정확하게 답변하세요. ###질문: {question} \n ###답변: "
    template = """
    ### Role ###
    You are an AI assistant who provides accurate and concise 'answers' to 'questions' in financial documents.
    Generate 'answers' to 'questions' based on the content of the given 'Document'.

    ### Document ###
    {context}

    ### Question ###
    {question}

    ### Answer ###

    """
    prompt = PromptTemplate.from_template(template)

    # RAG 체인 정의
    if context != "":
        rag_chain = (
            RunnableParallel(context=lambda x: x["context"], question = lambda x: x["question"])
            | prompt
            | llm
            | StrOutputParser()
        )
    else:
        rag_chain = (
            {"question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

    # 답변 추론
    print(f"Question: {question}")
    full_response = rag_chain.invoke({"question": question, "context": context})

    print(f"Answer: {full_response}\n")

    # 결과 저장

    if context != "":
        results.append({
            "Context": context,
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })
    else:
        results.append({
            "Question": question,
            "Answer": full_response,
            "True_Answer": eval_dataset['answer'][idx]
        })

Answering Questions:   0%|          | 0/98 [00:00<?, ?it/s]

Question: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?


Answering Questions:   1%|          | 1/98 [00:42<1:09:09, 42.78s/it]

Answer: 2,300,000백만원
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

Question: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?


Answering Questions:   2%|▏         | 2/98 [01:25<1:08:33, 42.85s/it]

...

Answer: 국가결산보고서는 '우발사항'으로 공시하는 반면, 지방자치단체 회계기준은 여전히 '우발부채'로 정의하고 있습니다.
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        


Question: 우발부채란 무엇이며, 그 관리가 왜 중요한가?


Answering Questions:  99%|█████████▉| 97/98 [1:08:22<00:42, 42.28s/it]

Answer: 우발부채는 미래에 특정 사건이 일어나지 않는 한 발생하지 않는 의무로, 하나 또는 그 이상의 조건이 충족되어야 발생한다.
        
        우발부채는 '잠재적 의무' 또는 '현재의무'로 구분된다.
        
        - '잠재적 의무'는 과거의 거래나 사건으로 발생하였지만, 현재의 경제적 편익이 있는 자원을 유출할 의무 여부가 확신되지 않은 상태를 의미한다.
        - '현재의무'는 과거의 거래나 사건으로 발생하였지만, 경제적 편익이 있는 자원의 유출 가능성이 낮거나 금액을 신뢰할 수 없어 부채로 인식하지 않는 경우이다.
        
        지속 가능한 재정 운영

Question: 보증이란 무엇이며, 어떤 형태의 보증이 재정상태표에 부채로 기록되는가? 또한 표준화 보증이란 무엇이며, 어떤 목적으로 발행되는가?


Answering Questions: 100%|██████████| 98/98 [1:09:05<00:00, 42.30s/it]

Answer: 보증은 어려운 상황에 대비하여 일정 금액을 지급할 의무를 지는 것을 의미합니다. 모든 보증이 재정상태표에 부채로 기록되는 것은 아니다. 하지만 파생금융상품의 형태로 이루어진 보증이나 표준화 보증 제도에서 설정하는 충당부채의 형태로 이루어진 보증은 재정상태표에 부채로 기록됩니다.
        
        
        표준화 보증은 통상 아주 적은 금액에 대해 획일적 조건으로 대규모로 발행하는 보증을 말합니다. 수출(무역)신용 보증, 환보증 등이 대표적인 예시이며, 이는 재정상태표에 부채로 기록됩니다.
        
        
        
        
        
        
        



In [ ]:
# ## 원문

# template = """
# 다음 정보를 바탕으로 질문에 답하세요:
# {context}

# 질문: {question}

# 주어진 질문에만 답변해주세요. 문장으로 답변해주세요. 답변할 때 질문의 주어를 써주세요.
# 답변:
# """

In [ ]:
# # 변경 2
# template = """
# ###Instruction###
# Instruction1:
# 1. You must answer the questions based on the following information: {context}

# 2. When answering, you must respond in "Korean".

# ###Example###
# Example1:
# "장애인 고용장려금은 2023년 대비 2024년에 어떻게 변화했는가?"
# (Answer:"장애인 고용장려금은 2023년에 2,977억원이었고, 2024년에 3,234억원으로 증가했다.")

# Example2:
# "어떤 국가들을 대상으로 인도적 지원을 확대할 것이며, 어떤 위험에 노출된 국가들을 주로 지원할 계획인가요?"
# (Answer:"우크라이나 등 분쟁지역과, 기후변화·식량위기 등 위험에 노출된 취약국을 대상으로 인도적 지원을 확대할 것이다.")


# ###Question###
# {question}
# """

In [ ]:
i = 4
print(eval_dataset['question'][i])
print('')
print(eval_dataset['context'][i])

혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?

2020. 1
미래기술육성자금, 고성장촉진자금 신규 지원
  2022. 1
미래기술육성자금 및 고성장촉진자금 사업 폐지
  2023. 1
창업기반지원과 신청 대상이 중복인 일자리창출촉진자금을 폐지 
  2023. 1
성장공유형 대출방식 추가 (혁신창업사업화자금)
< 국정과제 >
< 국정목표 2. 민간이 끌고 정부가 미는 역동적 경제 >
6. 중소/벤처기업이 경제의중심에 서는 나라
사   업   명
혁신창업사업화자금(융자) (5152 301)
1. 사업 코드 정보
구분
기금
소관
실국(기관)
계정
분야
부문
코드
중소벤처기업
창
업
 및
 진
흥
기
금
중소벤처기업부중소기업정책실
110
118
명칭
산업/중소기업 
및 에너지
창업 및 벤처
구분
프로그램
단위사업
세부사업
코드
5100
5152
301
명칭
창업환경조성
창업기업지원융자(기금)
혁신창업사업화자금(융자)
2. 사업 지원 형태 및 지원율
직접
출자
출연
보조
융자
총사업비 : 해당없음
  사업기간 : ’98년~ 계속
  사업규모: ‘24년 2조 78억원
  사업시행방법 : 융자, 보조
  사업시행주체 : 중소벤처기업진흥공단
  사업 수혜자 : 중소기업
  보조, 융자, 출연, 출자 등의 경우 보조/융자 등 지원 비율 및 법적근거
내역사업명
구분
피보조․피출연 
등 기관명
지원 금액
(2024 계획안)
지원 비율(%)
보조율 법적근거 
(해당 조항)
창업기반지원
융자
중소기업



# Evaluation

In [ ]:
#검증 데이터 쓸때만 사용 가능

import collections
# from SQUAD, 메트릭 계산 함수
def compute_f1(y_true, y_pred):
    common = collections.Counter(y_true) & collections.Counter(y_pred)
    num_same = sum(common.values())
    precision = 1.0 * num_same / len(y_pred) if len(y_pred) != 0 else 0
    recall = 1.0 * num_same / len(y_true) if len(y_true) != 0 else 0
    if len(y_pred) == 0 or len(y_true) == 0:
        f1 = int(y_pred == y_true)
    elif num_same == 0:
        f1 = 0
    else:
        f1 = (2 * precision * recall) / (precision + recall)
    return f1, precision, recall

for item in results:
    yhat = item["Answer"]
    y = item["True_Answer"]
    f1, precision, recall = compute_f1(y, yhat)
    item["F1"] = f1
    item["Precision"] = precision
    item["Recall"] = recall



In [ ]:
# 제출용 샘플 파일 로드
eval_df = pd.DataFrame([])

# 생성된 답변을 제출 DataFrame에 추가
eval_df['Question'] = [item['Question'] for item in results]
eval_df['Answer'] = [item['Answer'] for item in results]
eval_df["F1"] = [item["F1"] for item in results]
eval_df["Precision"] = [item["Precision"] for item in results]
eval_df["Recall"] = [item["Recall"] for item in results]
# eval_df['Answer'] = eval_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
eval_df.to_csv(f"{path}epoch_eval_prompt_ver3.csv", encoding='UTF-8-sig', index=False)

In [ ]:
# 평균 F1 확인
# path = '/content/drive/MyDrive/DACON/Finance/'

# eval_df = pd.read_csv(f"{path}trained_eval.csv",index_col=0)
eval_df["F1"].mean()

0.0

# Submission

In [ ]:
# 제출용 샘플 파일 로드
submit_df = pd.read_csv(f"{path}sample_submission.csv")

# 생성된 답변을 제출 DataFrame에 추가
# submit_df['Question'] = [item['Question'] for item in results]
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")     # 모델에서 빈 값 (NaN) 생성 시 채점에 오류가 날 수 있음 [ 주의 ]

# 결과를 CSV 파일로 저장
submit_df.to_csv(f"{path}gemma2_base_alltextv2_1epoch_prompt_ver3.csv", encoding='UTF-8-sig', index=False)

In [ ]:
# 질문/문맥/정답이 다 포함된 비교용 csv 저장
submit_df['Question'] = [item['Question'] for item in results]
submit_df['Context'] = [item['Context'] for item in results]
submit_df['Answer'] = [item['Answer'] for item in results]
submit_df["Answer"] = submit_df["Answer"].apply(str.rstrip)

submit_df.to_csv(f"{path}gemma2_base_alltextv2_1epoch_prompt_2_ver3.csv", encoding='UTF-8-sig', index=False)